#### wg 5 assignment
# Anomaly Detection

(This code brances off of committ 29821ccffaca5817b3fe0b3ae2d972f8132b7999.)

(As an aside, I'm using python 3, not 2. Only change that had to be made was on the print statement at [*])

Imports

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

Read data.

In [2]:
header = pd.read_table("kddcup.names.txt", header=None)
att_types = pd.read_table("training_attack_types.txt", sep=" ", header=None)

tr_raw = pd.read_csv("kddcup.data_10_percent_corrected", header=None)
test_raw = pd.read_csv("kddcup_testdata.corrected", header=None)

Do the preprocessing.

In [3]:
def preprocess(dat):
    dat.columns = header[0]
    att_types.columns = ["attack", "type"]
    dat["type"] = np.nan
    for i in range(0, len(att_types["attack"])):
        dat.loc[dat["attack"] == att_types.loc[i,].attack, "type"] = att_types.loc[i,].type
    dat.type = dat.type.fillna("unlisted")
    dat.attack = dat.attack.astype('category')

    dat[dat.select_dtypes(include=['number']).columns] = preprocessing.scale(
        dat[dat.select_dtypes(include=['number']).columns])
    dat = pd.get_dummies(dat, columns=['protocol_type', 'service', 'flag'])

    return dat

In [4]:
tr = preprocess(tr_raw)
test = preprocess(test_raw)  # actual test data (but labeled--don't cheat!)

Prepare for classification.

In [5]:
# not occurring in training set but occuring in test set so we just add it
tr['service_icmp'] = 0
test['service_red_i'] = 0
test['service_urh_i'] = 0

In [6]:
tr_labels = tr["type"].values
tr_features = tr.drop(["type", "attack"], axis=1).values

In [7]:
test_labels = test["type"].values
test_features = test.drop(["type", "attack"], axis=1).values

In [10]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
clf.fit(tr_features, tr_labels)
print(clf.score(test_features, test_labels)) # [*]

0.966704069395
